In [ ]:
import numpy as np
import pandas as pd
import uproot
import awkward_pandas
import matplotlib.pyplot as plt
import os
import math
import subprocess
import time
import itertools
import re

In [ ]:
def getGarfieldPath():
    """
    Reads and returns the filepath to the Garfield++ source script.
    
    Attempts to read the path to the Garfield++ source script given in 
    'GARFIELDPATH'. If this path-file does not exist, it creates one with a 
    placeholder message. If the path read from the file does not
    point to a valid 'setupGarfield.sh' file, an error message is printed.
    
    Returns:
        str or None: The validated Garfield++ source path if successful,
                     otherwise None.
    """
    filename = 'GARFIELDPATH'
    try:
        with open(filename, 'r') as file:
            garfieldPath = file.read().strip()
            if not os.path.exists(garfieldPath):
                print(f"Error: File 'setupGarfield.sh' not found at {garfieldPath}.")
                return None
            return garfieldPath

    except FileNotFoundError:
        print(f"Error: File '{filename}' was not found.")
        with open(filename, "w") as file:
            file.write('<<< Enter Garfield source path here. >>>')
            print(f"File '{filename}' created. Please update.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None


In [ ]:
def setupSimulation():
    """
    Initializes Garfield++ and creates an avalanche executable.
    
    Reads the Garfiled++ source path, and ensures a build directory.
    Compiles the executable using cmake and make.
    Initializes a simulation run counter if it does not already exist.

    Note: If a degmentation fault occurs, it is most likely that the
          Garfield++ library is not sources correctly.

    Returns:
        bool: True if the setup is successful, False otherwise.
    """
    #Get local Garfield path
    garfieldPath = getGarfieldPath()
    if garfieldPath is None:
        print('Error with Garfield source path.')
        return False

    #Check for build/
    if not os.path.exists("build"):
        os.makedirs("build")
    
    #Make executable
    makeBuild = (
        f'source {garfieldPath} && '
        f'cd build && '
        f'cmake .. && '
        f'make'
    )
    result = subprocess.run(
        makeBuild,
        shell=True,
        check=True,
        executable='/bin/bash',
        capture_output=True,
        text=True
    )

    #Check for run number file
    if not os.path.exists('runNo'):
        with open('runNo', 'w') as file:
            file.write('1000')
            
    return True

In [ ]:
def defaultParam():
    """
    Default FIMS parameters.
    Dimensions in microns.

    Returns:
        dict: Dictionary of default parameters and values.
    """
    defaultParam = {
        'padLength': 65.,
        'pitch': 250.,
        'gridStandoff': 100.,
        'gridThickness': 1.,
        'holeRadius': 16.,
        'cathodeHeight': 400.,
        'thicknessSiO2': 5.,
        'fieldRatio': 60.,
        'numFieldLine': 200,
        'transparencyLimit': .1,
        'numAvalanche': 1000,
        'avalancheLimit': 200,
        'gasCompAr': 80.,
        'gasCompCO2': 20.,
    }
    return defaultParam

In [ ]:
def readParam():
    """
    Reads and returns the simulation parameters contained 
    in the simulation control file.

    Returns:
        dict: Dictionary containing the parameters and values. 
              Empty if an error occurs.
    """
    filename = 'runControl'
    param = {}
    
    try:
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()  # Remove leading/trailing whitespace
                if line.startswith('/') or not line:  # Skip comments and empty lines
                    continue
                # Split the line at the '='
                parts = line.split('=', 1)
                if len(parts) == 2:
                    key, value = parts[0].strip(), parts[1].strip()
                    value = value.rstrip(';') # Remove trailing semicolon
                    param[key] = value
                else:
                    print(f"Skipping malformed line: {line}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return {}
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

    return param

In [ ]:
def writeFile(filename, lines):
    """
    Writes a list of strings to a specified file.
    Each string in the list is treated as a new line.

    Args:
        filename (str): The path to the file to write.
        lines (list): A list of strings to be written.
        
    Returns:
        bool: True if file written successfully, False otherwise.
    """
    try:
        with open(filename, 'w') as file:
            file.writelines(lines)
            
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return False
        
    return True

In [ ]:
def writeFile(filename, lines):
    """
    Writes a list of strings to a specified file.
    Each string in the list is treated as a line.

    Returns:
        bool: True if writing was successful, False otherwise.
    """
    try:
        with open(filename, 'w') as file:
            file.writelines(lines)
            
    except Exception as e:
        print(f"An error occurred while writing to {filename}: {e}")
        return False
        
    return True
    

In [ ]:
def writeRunControl(param=None):
    """
    Rewrites the simulation control file with the given input parameters.

    First ensures that values exist for all expected parameters,
    then updates them in the 'runControl' file.
    
    Args:
        param (dict): Dictionary containing the parameters
        to be written to the simulation control file.

    Returns:
        bool: True if new file written successfully, False otherwise.
    """
    filename = 'runControl'

    if param is None or not param:
        print(f"No parameters to write to {filename}.")
        return False

    allParam = defaultParam()
    for inParam in allParam:
        if inParam not in param:
            print(f"Missing parameter: '{inParam}'")
            return False

    #Read the old runControl file
    try:
        with open(filename, 'r') as file:
            oldLines = file.readlines()  # Read all lines of the file
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return False
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return False

    #Replace the old parameters with those in param
    newLines = []
    for line in oldLines:
        line = line.strip()
        if line.startswith('/') or not line:
            newLines.append(line + '\n')  # Keep comments and empty lines
            continue

        parts = line.split('=', 1)
        if len(parts) == 2:
            key = parts[0].strip()
            if key in param:
                newLines.append(f"{key} = {param[key]};\n")  # Update value
            else:
                newLines.append(line + '\n') #keep original line
        else:
            newLines.append(line + '\n')  # Keep non-parameter lines

    #Write new runControl file
    return writeFile(filename, newLines)

In [ ]:
def readSIF():
    """
    Reads the FIMS.sif file and returns its content as a list of lines.
    This file is assumed to be in the 'Geometry/' folder.

    Returns:
        list: A list of strings, each a line of FIMS.sif.
              Returns None if an error occurs.
    """
    filename = os.path.join('./Geometry', 'FIMS.sif')

    try:
        with open(filename, 'r') as file:
            sifLines = file.readlines()  # Read all lines of the file
            
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None 
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

    return sifLines

In [ ]:
def calcPotentials(param=None):
    """
    Calculates the required potentials to achieve a desired field ratio.

    Uses the geometry definitions and desired amplification field ratio
    contained in param. Assumes a drift field of 1 kV/cm.

    Args:
        param (dict): Dictionary containing the parameters of the simulation.

    Returns:
        dict: Dictionary containing the potentials for the cathode and grid.
              Empty if necessary parameters are unavailable. 
    """
    if param is None or not param:
        print(f"Invalid parameters.")
        return {}

    allParam = defaultParam()
    for inParam in allParam:
        if inParam not in param:
            print(f"Missing parameter: '{inParam}'")
            return {}
        
    convertEField = 0.1 # 1 kV/cm = 0.1 V/micron
    
    # Calculate the voltage required to achieve amplification field
    gridDistance = float(param['gridStandoff']) - float(param['gridThickness'])/2. #micron

    gridVoltage = float(param['fieldRatio'])*convertEField*gridDistance

    # Calculate for drift field
    cathodeDistance = float(param['cathodeHeight']) - float(param['gridThickness'])/2. #micron
    cathodeVoltage = convertEField*cathodeDistance + gridVoltage

    potentials = {
        'cathodeVoltage': -cathodeVoltage,
        'gridVoltage': -gridVoltage
    }
    
    return potentials

In [ ]:
def writeSIF(param=None):
    """
    Rewrites the FIMS.sif file boundary conditons based on the given parameters.

    Assumes that 'Potential' is defined on the line following 'Name'.

    Args:
        param (dict): Dictionary containing the parameters of the simulation.

    Returns:
        bool: True if file written successfully, False otherwise.
    """
    if param is None or not param:
        print(f"Invalid parameters.")
        return False

    allParam = defaultParam()
    for inParam in allParam:
        if inParam not in param:
            print(f"Missing parameter: '{inParam}'")
            return False

    #Read old .sif file
    sifLines = readSIF()
    if not sifLines:
        print('An error occurred while reading sif file.')
        return False

    potentials = calcPotentials(param)

    writeCathode = -1
    writeGrid = -1
    
    # Find the cathode and grid naming lines
    for i, line in enumerate(sifLines):
        if 'Name = "Cathode"' in line:
            writeCathode = i+1
        if 'Name = "Grid"' in line:
            writeGrid = i+1

    if writeCathode == -1 or 'Potential =' not in sifLines[writeCathode]:
        print('Error with cathode.')
        return False
    if writeGrid == -1 or 'Potential =' not in sifLines[writeGrid]:
        print('Error with grid.')
        return False

    #rewrite appropriate lines
    sifLines[writeCathode] = f'\tPotential = {potentials['cathodeVoltage']}\n'
    sifLines[writeGrid] = f'\tPotential = {potentials['gridVoltage']}\n'

    #Write new .sif file
    filename = os.path.join('./Geometry', 'FIMS.sif')
    return writeFile(filename, sifLines)

In [ ]:
def makeWeighting():
    """
    Writes a new .sif file for determining the weighting field.

    Sets all electrode boundary conditions to 0, then sets the pad potential to 1.

    Returns:
        bool: True if file written successfully, False otherwise.
    """
    #Read original sif file
    sifLines = readSIF()
    if not sifLines:
        print('An error occurred while reading sif file.')
        return False

    #Convert to a single string
    sifString = "".join(sifLines)

    #Replace all 'FIMS' with 'FIMSWeighting'
    sifString = sifString.replace('FIMS', 'FIMSWeighting')

    #Change all lines with 'Potential = <number>' to 'Potential = 0.0'
    # This regex matches 'Potential = ' followed by any number (integer or float, positive or negative)
    sifString = re.sub(r'Potential = [-+]?\d+\.?\d*', r'Potential = 0.0', sifString)

    #Handle the 'CentralPad' case separately and set its Potential to 1.0
    # Note: This is only for the middle Pad. The 'SurroundingPads' case can be handled by replacing the 3's with 4's.
    sifString = sifString.replace(
        'Boundary Condition 3\n Target Boundaries(1) = 3\n Name = "CentralPad"\n \tPotential = 0.0',
        'Boundary Condition 3\n Target Boundaries(1) = 3\n Name = "CentralPad"\n \tPotential = 1.0'
    )

    #Convert back to a list
    sifLinesNew = sifString.splitlines(keepends=True)
    
    #Write new sif file
    filename = 'FIMSWeighting.sif'
    return writeFile(filename, sifLinesNew)

In [ ]:
def writeParam(param=None):
    """
    Updates the simulation control files with the specified parameters.

    Validates input params, then writes simulation files.

    Returns:
        bool: True if all write operations were successful, False otherwise.
    """
    if param is None or not param:
        print(f"No parameters to write.")
        return False

    allParam = defaultParam()
    for inParam in allParam:
        if inParam not in param:
            print(f"Missing parameter: '{inParam}'")
            return False
        
    if not writeRunControl(param):
        return False
    if not writeSIF(param):
        return False
    
    return True

In [ ]:
def resetParam(verbose=True):
    """
    Rewrites the run control files with the default simulation parameters.

    Args:
        verbose (bool): Option available to supress reset notification.

    Returns:
        dict: Dictionary containing the default simulation parameters.
              Returns None if unable to write the defaults to files.
    """
    defaultParams = defaultParam()
    if not writeParam(defaultParams):
        print('Error resetting parameters.')
        return None

    if verbose:
        print('Parameters have been reset.')
    
    return defaultParams

In [ ]:
def getRunNumber():
    """
    Gets the simulation number for the NEXT simulation.

    This number is stored in 'runNo'.

    Return:
        int: The simulation run number. Returns -1 if an error occurs.
    """
    filename = 'runNo'

    try:
        with open(filename, 'r') as file:
            content = file.read().strip()
            runNo = int(content)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return -1
    except ValueError:
        print(f"Error: Invalid number format in '{filename}")
        return -1
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return -1
    
    return runNo

In [ ]:
def runGmsh():
    """
    Runs the Gmsh program to generate a 3D finite-element mesh of the simulation geometry.
    Writes the output of Gmsh to 'log/logGmsh'.

    Utilizes several Gmsh options, including:
        -order 2: Second-order meshing
        -optimize_ho: Optimize the higher-order mesh
        -clextend: Extends the characteristic lengths to the whole geometry
        OptimizeNetgen: Enables Netgen algorithm optimization
        MeshSizeFromPoints: Uses mesh sizes defined at specific points

    Returns:
        bool: True if Gmsh runs successfully, False otherwise.
    """
    try:
        geoFile = 'FIMS.txt'
        with open(os.path.join(os.getcwd(), 'log/logGmsh.txt'), 'w+') as gmshOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['gmsh', os.path.join('./Geometry/', geoFile),
                 '-order', '2', '-optimize_ho',
                 '-clextend', '1',
                 '-setnumber', 'Mesh.OptimizeNetgen', '1',
                 '-setnumber', 'Mesh.MeshSizeFromPoints', '1',
                 '-3',
                 '-format', 'msh2'],
                stdout=gmshOutput, 
                check=True
            )
            endTime = time.monotonic()
            gmshOutput.write(f'\n\nGmsh run time: {endTime - startTime} s')

            if runReturn.returncode != 0:
                print('Gmsh failed. Check log for details.')
                return False
                
    except FileNotFoundError:
        print("Unable to write to 'log/logGmsh.txt'.")
        return False
        
    return True


In [ ]:
def runElmer():
    """
    Runs Elmer to determine a finite-element Electric field solution.

    Converts a gmsh mesh to elmer format using ElmerGrid.
    Calculates potentials and E fields for the mesh using ElmerSolver.
    Output files are saved to a subdirectory called 'elmerResults/'.
    Writes the output of the programs to 'log/logElmerGrid' and 'log/logElmerSolver'.

    Returns:
        bool: True if ElmerGrid and ElmerSolver both run successfully.
              False otherwise.
    """
    originalCWD = os.getcwd()
    os.chdir('./Geometry')

    os.makedirs("elmerResults", exist_ok=True)
        
    try:
        with open(os.path.join(originalCWD, 'log/logElmerGrid.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['ElmerGrid', '14', '2', 'FIMS.msh', 
                 '-names',
                 '-out', 'elmerResults', 
                 '-autoclean'], 
                stdout=elmerOutput,
                check=True
            )
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerGrid run time: {endTime - startTime} s')
            
            if runReturn.returncode != 0:
                print('ElmerGrid failed. Check log for details.')
                return False
            
        with open(os.path.join(originalCWD, 'log/logElmerSolver.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['ElmerSolver', 'FIMS.sif'],
                stdout=elmerOutput,
                check=True
            )
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerSolver run time: {endTime - startTime} s')

        if runReturn.returncode != 0:
                print('ElmerSolver failed. Check log for details.')
                return False
    finally:
        os.chdir(originalCWD)
    return True

In [ ]:
def runElmerWeighting():
    """
    Runs ElmerSolver to determine the weighing field for a simulation.

    Assumes that the Gmsh mesh has already been converted to 
    Elmer format by ElmerGrid. Creates the appropriate .sif file.
    Writes the ElmerSolver output to 'log/logElmerSolverWeighting'.

    Returns:
        bool: True if ElmerSolver runs successfully, False othwerwise.
    """
    if not makeWeighting():
        print("Error occured creating weighting '.sif' file.")
        return False
    
    originalCWD = os.getcwd()
    os.chdir('./Geometry')
    try:
        with open(os.path.join(originalCWD, 'log/logElmerSolverWeighting.txt'), 'w+') as elmerOutput:
            startTime = time.monotonic()
            runReturn = subprocess.run(
                ['ElmerSolver', 'FIMSWeighting.sif'],
                stdout=elmerOutput, 
                check=True
            )
            endTime = time.monotonic()
            elmerOutput.write(f'\n\nElmerSolver run time: {endTime - startTime} s')

        if runReturn.returncode != 0:
                print('ElmerSolver failed for weighting. Check log for details.')
                return False
    finally:
        os.chdir(originalCWD)
    return True

In [ ]:
def runGarfield():
    """
    Runs a Garfield++ executable to determine field lines and simulate 
    electron avalanches based on the parameters in 'runControl'.

    First links garfield libraries, creates the executable, and then runs the simulation.
    The simulation is numbered based on the number found in 'runNo';
    This also incremenmts this number.
    The information from this simulation is saved in .root format within 'Data/'.
    
    Returns:
        bool: True if Garfield executable runs successfully, False otherwise.
    """
    originalCWD = os.getcwd()
    garfieldPath = getGarfieldPath()
    os.chdir('./build/')
    try:
        with open(os.path.join(originalCWD, 'log/logGarfield.txt'), 'w+') as garfieldOutput:
            startTime = time.monotonic()
            setupAvalanche = (
                f'source {garfieldPath} && '
                f'make && '
                f'./runAvalanche'
            )
            runReturn = subprocess.run(
                setupAvalanche, 
                stdout=garfieldOutput, 
                shell=True, 
                check=True
            )
            endTime = time.monotonic()
            garfieldOutput.write(f'\n\nGarfield run time: {endTime - startTime} s')

        if runReturn.returncode != 0:
                print('Garfield++ execution failed. Check log for details.')
                return False
    finally:
        os.chdir(originalCWD)
    return True

In [ ]:
def runSimulation(param=None, changeGeometry=True):
    """
    Executes the full simulation process for the given parameters.
    Resets parameters upon completion.

    Simulation process:
        1. Check that all required parameters are present in 'param'.
        2. Read the run number in 'runNo'.
        3. Write the simulation parameters to the control files.
        4. Execute Gmsh to generate a finite-element mesh of the geometry.
        5. Execute Elmer to solve the E field for the mesh.
        6. Execute Elmer to solve the weighting field for the electrode.
        7. Execute the Garfield++ execuatble simulation.

    Args:
        param (dict): The simulation parameters.
        changeGeometry (bool): Allows for bypassing bypassing the 
                               Gmsh call to generate a mesh.
                               (Optional for when geometry does not change.)

    Returns:
        int: The run number of the simulation that was executed. 
             Returns -1 if any errors occur.
    """

    if param is None or not param:
        print('No parameters to run.')
        return -1
        
    allParam = defaultParam()
    for inParam in allParam:
        if inParam not in param:
            print(f"Missing parameter: '{inParam}'")
            return -1

    #get the run number for this simulation
    runNo = getRunNumber()
    if runNo == -1:
        print("Error reading 'runNo'")
        return -1
    print(f'Running simulation - Run number: {runNo}')
    
    #write parameters for sim
    if not writeParam(param):
        print('Error writing parameters.')
        return -1

    #Allow for skipping gmsh if geometry has not changed.
    if changeGeometry:
        if not runGmsh():
            print('Error executing Gmsh.')
            return -1

    #Determine the Electric and weighting fields
    if not runElmer():
            print('Error executing Elmer (base).')
            return -1     
    if not runElmerWeighting():
        print('Error executing Elmer (weighting).')
        return -1

    #Run the electron transport simulation
    if not runGarfield():
        print('Error executing Garfield.')

    #reset parameters to finish
    _ = resetParam()
    
    return runNo

In [ ]:
def scanParam(parameters, minValues, maxValues, numSteps):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameters (list): List of strings for parameters to be scanned.
        minValues (list): List of numbers; Minimum value of each parameter.
        maxValues (list): List of numbers; Maximum value of each parameter.
        numSteps (list): List of ints; Number of steps within each parameter's range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """

    if not (len(parameters) == len(minValues) == len(maxValues) == len(numSteps)):
        raise ValueError('Inputs must have the same length.')

    #Reset parameters before scan
    allParam = resetParam()

    for inParam in parameters:
        if inParam not in allParam:
            print(f"Parameter '{inParam}' not able to be varied.")
            return []
    
    #Find range for each parameter
    allRanges = []
    for i, _ in enumerate(parameters):
        paramRange = np.linspace(minValues[i], maxValues[i], numSteps[i])
        allRanges.append(paramRange)

    print(f'Beginning scan through {parameters}. {math.prod(numSteps)} runs. Initial run: {getRunNumber()}')

    runNos = []
    for scanParams in itertools.product(*allRanges):

        #Reset parameters to defaults
        param = resetParam(verbose=False)
        #Apply current parameter combination
        for i, inParam in enumerate(parameters):
            param[inParam] = scanParams[i]

        runNo = runSimulation(param)
        if runNo == -1:
            print(f'Simulation failed with settings: {param}')
        runNos.append(runNo)

    print(f'Done scan through {parameters}. Final run number: {runNos[-1]}')
    _ = resetParam()
        
    return runNos

In [ ]:
def scanValues(parameterDefinitions):
    """
    Executes a parameter scan by running the simulation multiple times with
    varying combinations of specified parameters.

    Executes an N-dimensional linear scan with every combination of parameters.
    If a single simulation fails, its run number is set to -1 and the scan continues.

    Args:
        parameterDefinitions (list): List of the following form: 
                                     ['parameter', minValue, maxValue, stepValue]
                                     'parameter' - String of the parameter name.
                                      minValue - Number for the minimum parameter value.
                                      maxValue - Number for the maximum parameter value.
                                      stepValue - Integer number of steps through range.

    Returns:
        list: List of the run numbers of the executed simulations.
              Empty if the parameters are not recognized.
    """
    #Separate the elements of scanning parameters
    parameters = []
    minValues = []
    maxValues = []
    numSteps = []
    
    for inDefinition in parameterDefinitions:
        if not (len(inDefinition) == 4):
            raise ValueError("Input list elements must have the form: ['parameter', minValue, maxValue, stepValue]")
        
        inParam, inMin, inMax, inStep = inDefinition
            
        parameters.append(inParam)
        minValues.append(inMin)
        maxValues.append(inMax)
        numSteps.append(inStep)

    #run the scan
    runNos = scanParam(parameters, minValues, maxValues, numSteps)
        
    return runNos

In [ ]:
def readRootTrees(runNumber: int) -> dict:
    """
    Reads all trees from a ROOT file and returns them as a dictionary.
    The keys are the tree names and the values are pandas DataFrames.

    Args:
        runNumber (int): The run number of the ROOT file.

    Returns:
        dict: A dictionary where keys are tree names (str) and values are pandas DataFrames. 
              Empty if the file cannot be opened or contains no trees.
    """
    
    #TODO - This function itself is named similarly to one in analysis.ipynb
    #    They rely on the location of the data folder.
    #    Likely solution - move data up the heirarchy, to exist at similar relative place. (../../ i think)
    dataFilePath = 'Data/' 
    dataFile = f'sim.{runNumber}.root'
    fullFilePath = os.path.join(dataFilePath, dataFile)
    
    try:
        with uproot.open(fullFilePath) as rootFile:
            dataframes = {}
            for treeName in rootFile.keys():
                if isinstance(rootFile[treeName], uproot.behaviors.TTree.TTree):
                    tree = rootFile[treeName]
                    try:
                        df = tree.arrays(library='pd')
                        dataframes[treeName] = df
                    except Exception as e:
                        print(f"Error reading tree '{treeName}': {e}")
            return dataframes
    except Exception as e:
        print(f"Error opening or reading ROOT file '{fullFilePath}': {e}")
        return {}

In [ ]:
def readMultipleRuns(runNumbers, treesToRead=None):
    """
    Reads the metatdata and desired simulation data from the specified runs.

    MetaData is always included in output. 
        If treesToRead is an empty list then only metadata is output.
        If treesToRead is None then all are output.
    The data is converted to pandas dataframes within a dictionary.
    The keys to this dict are the run numbers.

    Args:
        runNumbers (list): list of run numbers to be read.
        treesToRead (list): List of strings containing the desired tree name to read.

    Returns:
        allData (dict): Dictionary containing pandas dataframes with the simulation data.
    """
    #TODO - This function also exists in analysis.ipynb
    possibleDataTrees = [
        'fieldLineData',
        'gridFieldLineData',
        'electronData',
        'ionData',
        'avalancheData',
        'electronTrackData'
    ]

    filteredToRead = []
    if treesToRead is None:
        filteredToRead = possibleDataTrees
    elif not treesToRead:
        filteredToRead = []
    else:
        #Error check the tree parsing
        invalidTrees = []
        for inTree in treesToRead:
            if inTree == 'metaData':
                continue
            if inTree not in possibleDataTrees:
                invalidTrees.append(inTree)
            else:
                filteredToRead.append(inTree)
                
        if invalidTrees:
            raise ValueError(f"Cannot read the following trees: '{invalidTrees}'.")

    #Read in data for each run
    allData = {}
    for inRun in runNumbers:

        #Read the root tree for this run
        try:
            simData = readRootTrees(inRun)
        except FileNotFoundError:
            print(f"Warning: Could not find data for run {inRun} - Skipping.")
            continue

        allData[inRun] = {}

        #Always read the metadata, skip run if unsuccessful
        try:
            allData[inRun]['metaDataTree'] = simData['metaDataTree;1']
        except KeyError:
            print(f"Warning: 'metadata' not found in run {inRun}. Skipping this run.")
            allData[inRun]['metadata'] = None 
            continue

        #Read the remaining trees
        for inTree in filteredToRead:
            treeKey = f'{inTree}Tree;1'
            try:
                allData[inRun][inTree] = simData[treeKey]
            except KeyError:
                print(f"Warning: Tree '{inTree}' not found for run {inRun}.")
                allData[inRun][inTree] = None

    return allData

In [ ]:
def checkFieldTransparency(runNumber):
    """
    Checks the field transparency for a given simulation run number.
    Checks against the simulation's limit in its metadata.

    Args:
        runNumber (int): Run number.

    Returns:
        bool: True if tranparency is greater than the limit, False otherwise.
    
    """    
    simData = readMultipleRuns([runNumber], [])
    transparency = simData['metaData']['Field Transparency'].iloc[0]
    limit = simData['metaData']['Field Transparency Limit'].iloc[0]
    
    if transparency < limit:
        return False
    
    return True

In [ ]:
def transparencyScan(baseParam, scanParam, order=True, changeGeo=True):
    """
    Performs a 1-dimensional scan of the simulation,
    looking for the field transparency -limiting case.

    Sets all parameters based on baseParam, 
    then linearly scans through a specified parameter.
    If 'order' is True it scans in increasing order, descending if False.
    Simulations terminate if the transparency falls below the limit.

    Args:
        baseParam (dict): Dictionary containing
                          the base simulation parameters.
        scanParam (list): List of the form: 
                          ['parameterToScan', minValue, maxValue, numSteps]
                            - parameterToScan (str): The name of the parameter to vary.
                            - minValue (float): The minimum value for the scan.
                            - maxValue (float): The maximum value for the scan.
                            - numSteps (int): The number of steps in the scan.
        order (bool): Boolean that dictates the order of the scan. 
                      Ascending if True, descending if False.
        changeGeo (bool): Optional parameter to bypass geometry meshing.
                           Efficient if geometry does not change between runs.

    Returns:
        list: List of run numbers where the field transparency is greater than the limit.
              Empty if not all parameters are provided in baseParam or 
              if the scanning parameter is invalid
    """
    #Reset parameters before scan
    allParam = resetParam()

    for inParam in allParam:
        if inParam not in baseParam:
            print(f"Base parameter '{inParam}' not included.")
            return []

    if scanParam[0] not in allParam.keys():
        print('Invalid scanning parameter.')
        return []

    #Determine values for scan
    scanValues = np.linspace(scanParam[1], scanParam[2], scanParam[3])
    if not order:
        scanValues = np.flip(scanValues)
    
    runNos = []
    print(f'Beginning transparency scan for: {scanParam[0]}')
    print(f'\tTransparency limit is: {baseParam['transparencyLimit']}')
    for inValue in scanValues:
        baseParam[scanParam[0]] = inValue

        #Ensure geometry is generated on at least the first sim
        if inValue == scanValues[0]:
            runNo = runSimulation(baseParam)
        else:
            runNo = runSimulation(baseParam, changeGeo)

        if runNo == -1:
            print(f'An error occured at: {scanParam[0]} = {inValue}')
            break

        #Terminate scan if transparency is below limit
        if not checkFieldTransparency(runNo):
            print(f'Field transparency too low at: {scanParam[0]} = {inValue}. Ending scan')
            break

        runNos.append(runNo)

    #Done running - reset to defaults
    print(f'Done scan through {scanParam[0]}. Final run number: {getRunNumber()-1}')
    _ = resetParam()
    
    return runNos

In [ ]:
# Example test run of a simulation.
## Sets up simulation.
## Utilizes the default parameters.
## Determines a start-to-finish execution time.
if __name__ == '__main__':
    setupSimulation()
    startTime = time.monotonic()
    runSimulation(resetParam())
    endTime = time.monotonic()
    print(f'Test run takes ~{endTime-startTime:.1f}s.')

    '''
    Last time Tanner ran this test:
    Total time: ~960 s
        Gmsh: ~17 s
        ElmerGrid: <1 s   
        ElmerSolver: ~351 s
        ElmerWeighting: ~16 s
        Garfield: ~547 s
    '''